In [ ]:
import random
import numpy as np
import pandas as pd

# Music Practice Builder
- Load data
- create empty practice session
- split into essential and non-essential items
- add essential items to session
- sort by priority


## Configuration

In [ ]:
input_file = './practice_elements.xlsx'
practice_time_minutes = 30

Boosting a category inflates the chance that items in that category will be selected for the session. The boost value determines the amount of overrepresentation.

The current implementation uses random sampling from the item pool, and so only integer boosts are allowed. Boosting is implemented by adding multiple copies of the given category. Later implementations may use a roulette wheel random selection, which will allow fractional boost values.

A value of 0 indicates that the category should not be boosted.

In [ ]:
category_boosters = {
    'pattern': 0,
    'technique': 0,
    'repertoire': 0,
    'sight reading': 2,
}

## Slicing and dicing the data

In [ ]:
data = pd.read_excel(
    input_file, 
    header=0,
    converters=
    {
        'min_time': int,
        'max_time': int,
        'priority': float,
        'essential': bool,
    })

In [ ]:
session = data.query('essential == True')

In [ ]:
items = data.query('essential == False')

Apply the category boosts

In [ ]:
for category, boost in category_boosters.items():
    if boost > 0:
        items_to_boost = items.query("category == '{0}'".format(category))
        items = items.append([items_to_boost] * boost, ignore_index=True)

## Generate the initial set of random times for the essential items

In [ ]:
times_df = pd.DataFrame({'time':session.apply(lambda row: random.randrange(row.min_time, row.max_time+1), axis=1)}, 
                        index=session.index)
session = session.join(times_df)

## Fill the rest of the session

In [ ]:
num_attempts_to_fill_time = 0
max_attempts_to_fill_time = 100
while session.time.sum() < practice_time_minutes and len(items) > 0:
    i = items.sample(n=1)
    items = items.drop(i.index)
    i['time'] = random.randrange(i.min_time.iloc[0], i.max_time.iloc[0]+1)
    if session.time.sum() + i.time.sum() <= practice_time_minutes:
        session = session.append(i)
    else:
        num_attempts_to_fill_time += 1
        if num_attempts_to_fill_time < max_attempts_to_fill_time:
            print(num_attempts_to_fill_time)
            continue
        else:
            print('Bailing out, may not have filled all time')
            break

# Today's Practice Session

In [ ]:
print('Target time: {0}, Actual time: {1}'.format(practice_time_minutes, session.time.sum()))

In [ ]:
session.sort_values(by='priority')[['name', 'category', 'time']]